In [ ]:
import os
import sys

os.chdir("..")
sys.path.append("../../")

# Perform likelihood ratio test

## Introduction

In this section, we will show how to use `perform_lrt` function to perform likelihood ratio test to compare two dicts of marginal models.

To get detailed information of the input and output of the function, please check [API](../set_up/_autosummary/scDesign3Py.perform_lrt.rst).

## Step 1: Import packages and Read in data

### import pacakges


In [ ]:
import anndata as ad
import scDesign3Py

### Read in data

The raw data is from the [scvelo](https://scvelo.readthedocs.io/scvelo.datasets.pancreas/) and we only choose top 30 genes to save time.

In [ ]:
data = ad.read_h5ad("data/PANCREAS.h5ad")
data = data[:, 0:30]
data

## Step 2: Run the `fit_marginal()` method to create the marginal models 

In [ ]:
# create the instance
test1 = scDesign3Py.scDesign3(n_cores=3, parallelization="mcmapply", return_py=False)
test1.set_r_random_seed(123)

# construct data
test1.construct_data(
    anndata=data,
    default_assay_name="counts",
    celltype="cell_type",
    pseudotime="pseudotime",
    corr_formula="cell_type",
)

# fit marginal
marginal1 = test1.fit_marginal(
    mu_formula="1",
    sigma_formula="1",
    family_use = "nb",
    usebam = False,
)

In [ ]:
# create the instance
test2 = scDesign3Py.scDesign3(n_cores=3, parallelization="mcmapply", return_py=False)
test2.set_r_random_seed(123)

# construct data
test2.construct_data(
    anndata=data,
    default_assay_name="counts",
    celltype="cell_type",
    pseudotime="pseudotime",
    corr_formula="pseudotime",
    ncell=10000,
)

# fit marginal
marginal2 = test1.fit_marginal(
    mu_formula="s(pseudotime, bs = 'cr', k = 10)",
    sigma_formula="1",
    family_use = "nb",
    usebam = False,
)

## Step 3: Get the marginal models from the `fit` key

In [ ]:
marg_test_1 = {key:value.rx2("fit") for key,value in marginal1.items()}
marg_test_2 = {key:value.rx2("fit") for key,value in marginal2.items()}

## Step 4: Run the `perform_lrt` function to perform the likelihood ratio test

The first argument is the alternative hypothesis and the second is the null hypothesis.

The return value is a `pandas.DataFrame` object with each row corresponding to a gene (marginal) model LRT result.

In [ ]:
scDesign3Py.perform_lrt(marg_test_2,marg_test_1).head()